# 1. PDF Parser

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
import re

In [2]:
#This part of the code sets up general default parameters associated with the PDFMiner library to loop and convert
#a PDF into text
def convert_pdf_to_txt(path, pages = None):
    if not pages:
        pagenos = set()
    else: pagenos = set(pages) 
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'big5'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    strng = retstr.getvalue()
    retstr.close()
    return strng

In [3]:
#This is a continuation of the code above specifying how to parse and save each line of code. 
#The main challenge was taking into account the number of '.' between each item on each line which is taken
#care of in the for loop
#Each line would ideally have 5 attributes: name, size, and then price of single, pack, and full. 
#To maintain continunity, we only kept the items that had a length of five as it was easier to manage when we 
#went on to create the mongodb database. 
path = "beer-corridorwine.pdf"
raw1 = convert_pdf_to_txt(path) #, pages = [1]
raw1 = filter(lambda x: x.strip(), raw1.split("\n"))
category = ['American Microbrews','Microbrews', 'Alternative Brews', 'Imports']
exclude = ['Non-Alcoholic Beer']
raw1 = [line for line in raw1 if ("..." in line and 'NA' not in line and 'N/A' not in line and 'Coke' not in line and 'Coca-Cola' not in line) or line in category]

data = []

for line in raw1:  
    line = re.sub(r'([a-z\.])\s{2}([0-9])', r'\1...\2', line, count = 1)
    line = re.sub(r'\s{2}\.{1}', '...', line, count = 1)
    line = re.sub(r'[\.]{2,}', ',', line, count= 1)
    line = re.sub(r'[\.]{42,}', ',,,', line)
    line = re.sub(r'[\.]{39,41}', ',,', line)  
    line = re.sub(r'[\.]{2,19}', ',', line)
    line = re.sub(r'\s\.', ',', line)
    line = re.sub(r'(pk)\s([0-9])', '\1,\2', line)
    line = re.sub(r'([0-9])\s([0-9])', '\1,\2', line)             
    line = line.split(',')
    length = []
    for d in line:
        i = line.index(d)
        if d != '' and d != ' ':
            line[i] = d.strip()
    if len(line) == 5:
        print line

['21st Amendment Back in Black', '12oz Can', '1.99', '8.99', '35.96']
['21st Amendment Brew Free! IPA', '12oz Can', '1.99', '8.99', '35.96']
['21st Amendment Watermelon Wheat', '12oz Can', '1.99', '8.99', '35.96']
['Abita Abbey Ale', '22oz Btl', '4.99', '', '']
['Abita Amber Lager', '12oz Btl', '1.49', '6.99', '27.96']
['Abita Andygator', '22oz Btl', '4.99', '', '']
['Abita Christmas Ale', '12oz Btl', '', 'Seasonal', '']
['Abita Golden', '12oz Btl', '1.49', '6.99', '27.96']
['Abita Jockamo IPA', '12oz Btl', '1.49', '6.99', '27.96']
['Abita Purple Haze', '12oz Btl', '1.49', '6.99', '27.96']
['Abita Restoration Pale Ale', '12oz Btl', '1.49', '6.99', '27.96']
['Abita Turbodog', '12oz Btl', '1.49', '6.99', '27.96']
['Abita Wheat Lager', '12oz Btl', '1.49', '7.99', '31.96']
['Allagash Black', '750ml Btl', '8.99', '', '']
['Allagash Curieux', '750ml Btl', '18.99', '', '']
['Allagash Dubbel Reserve', '12oz Btl', '3.79', '8.99-4pk', '53.94']
['Allagash Interlude', '750ml Btl', '21.99', '', '']

In [5]:
#After saving the prelimary results in a textfile, to make importing into a MongoDB database easier, 
#We parsed through the .txt to 1) edit and remove extraneous characters and 2) create a JSON array to directly import
#Additionally, we added in attributes of "alcohol type" and "country." 

import json



f = open("beer info.txt", "r")
f2 = open("scrapped_country.json","w")
f2.write("[")
for time in range(0,1149):
    if time < 626:
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("USA")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 626) and (time < 660):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("USA")
        info.append("Cider")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 660) and (time <= 703):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("USA")
        info.append("Alternative Brews")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 703) and (time < 726):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("USA")
        info.append("Cooler")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 726) and (time < 732):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Austria")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 732) and (time < 742):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Austria")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 742) and (time < 874):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Belgium")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 874) and (time < 899):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Canada")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 899) and (time < 902):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("China")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 902) and (time < 918):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Czech Republic")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 918) and (time < 976):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("England")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 976) and (time < 979):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("France")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 979) and (time < 1053):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Germany")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1053) and (time < 1054):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Greece")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1054) and (time < 1056):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("India")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1056) and (time < 1068):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Ireland")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1068) and (time < 1069):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Israel/ME")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1069) and (time < 1072):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Italy")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1072) and (time < 1080):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Japan and 'Far East'")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1080) and (time < 1087):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Latin America")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1087) and (time < 1104):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Mexico")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1104) and (time < 1111):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("New Zealand")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1111) and (time < 1113):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Poland")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1113) and (time < 1115):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Portugal")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1115) and (time < 1128):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Scotland")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1128) and (time < 1129):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Singapore")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1129) and (time < 1131):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Spain")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1131) and (time < 1134):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Vietnam and SE Asia")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        f2.write(",")
    if (time >= 1134) and (time < 1149):
        info = f.readline()
        info = info.strip("[").strip("]\n")
        info = info.replace("'", "")
        info = info.replace("\\xa1\\xa6","")
        info = info.replace("\\x020","")
        info = info.replace("\\x01","")
        info = info.replace("\\xa1\\xa","")
        info = info.replace("\\xa1\\xa5","")
        info = info.replace("\\x02","")
        info = info.replace("Dog sh", "Dogfish")
        info = info.replace("Schla y","Schlafly")
        info = info.replace("Af igem","Affligem")
        info = info.split(",")
        info.append("Popular Domestic Brews")
        info.append("Beer")
        titles = ["Name","Size","Single","Six_Pack","Full", "Country", "Alcohol_Type"]
        data = dict(zip(titles, info))
        f2.write(json.dumps(data))
        if time != 1148:        
            f2.write(",")
        
    
    
    
f2.write("]")

f2.close()
f.close()


# 2. Connect to MongoDB Server

In [9]:
from pymongo import MongoClient

In [12]:
client = MongoClient()
db = client.msa_database
coll1 = db.Price

In [13]:
cursor = coll1.find().limit(5)
for result in cursor:
    print(result)


{u'Pks': 8.99, u'Full': 35.96, u'Name': u'21st amendment back in black', u'Country': u'USA', u'Single': 1.99, u'Alcohol Type': u'Beer', u'_id': ObjectId('566606bd421d5349cc98c1f1'), u'Size': u'12oz Can'}
{u'Pks': 6.99, u'Full': 27.96, u'Name': u'abita amber lager', u'Country': u'USA', u'Single': 1.49, u'Alcohol Type': u'Beer', u'_id': ObjectId('566606bd421d5349cc98c1f2'), u'Size': u'12oz Btl'}
{u'Pks': None, u'Full': None, u'Name': u'abita andygator', u'Country': u'USA', u'Single': 4.99, u'Alcohol Type': u'Beer', u'_id': ObjectId('566606bd421d5349cc98c1f3'), u'Size': u'22oz Btl'}
{u'Pks': None, u'Full': None, u'Name': u'abita abbey ale', u'Country': u'USA', u'Single': 4.99, u'Alcohol Type': u'Beer', u'_id': ObjectId('566606bd421d5349cc98c1f4'), u'Size': u'22oz Btl'}
{u'Pks': 8.99, u'Full': 35.96, u'Name': u'21st amendment brew free! ipa', u'Country': u'USA', u'Single': 1.99, u'Alcohol Type': u'Beer', u'_id': ObjectId('566606bd421d5349cc98c1f5'), u'Size': u'12oz Can'}


--------------------------------------------------

In [14]:
#This shows the distribution of different beers by country 

import plotly.plotly as py
import pandas as pd

df = pd.read_csv('World_Map.csv')

data = [ dict(
        type = 'choropleth',
        locations = df['CODE'],
        z = df['Beer Types'],
        text = df['Country'],
        colorscale = [[0,"rgb(204, 255, 229)"],[(2/300),"rgb(153, 255, 1204)"],[0.01,"rgb(102, 255, 178)"],\
            [0.02,"rgb(51, 255, 153)"],[(10/300),"rgb(0, 255, 128)"],[0.04,"rgb(0, 204, 102)"],[0.08,"rgb(220, 220, 220)"],
        [0.2,"rgb(220, 220, 220)"] ,[0.4,"rgb(220, 220, 220)"],[0.8,"rgb(220, 220, 220)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = True,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            )
        ),
        colorbar = dict(
            autotick = False,
            tickprefix = '#',
            title = 'Beers Offered'
        ),
    ) ]

layout = dict(
    title = 'Distribution of Alcohol',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
url = py.plot( fig, validate=False, filename='d3-world-map' )



In [15]:
#This shows the distribution of beers by state

import plotly.plotly as py
import pandas as pd

df = pd.read_csv('US_Map.csv')

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df["State"]+": "+df['Count'] + " Breweries"




data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['Code'],
        z = df['Count'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "# of Breweries"
        )
    ) ]

layout = dict(
        title = 'Distribution of Breweries in the US',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )
    
fig = dict( data=data, layout=layout )

url = py.plot( fig, filename='us-cloropleth-map', validate=False )


# 3. Query

In [16]:
from pymongo import MongoClient
import pandas as pd
import sys

Connect to database

In [17]:
class Base:
    
    def __init__(self):
        self.client = MongoClient()
        self.db = self.client.msa_database
        self.cond = {}

Classes for Price, ABV and Rating

In [18]:
#The next lines of code are all dedicated to setting up the parameters for our query searches. 
#We have three different collections (Essentially tables). The 'class Price' allows for the user to search 
#by price range, country or name (full or partial) <-(This was parsed from the PDF) 


class Price(Base):
    
    def askCondition(self):
        self.cond = {'single_price': None, 'name': None}
        price = raw_input('Enter price range or hit enter: (e.g. 0, 2.29 ) ')
        countries = {}
        for i in range(len(self.db.Price.distinct('Country'))):
            countries[i] = self.db.Price.distinct('Country')[i]
        
        try: 
            if price != '':
                self.cond['single_price'] = map(float, price.replace(' ', '').split(','))    
                if len(self.cond['single_price']) != 2:
                    raise TypeError
            
            name = raw_input('Enter keyword of beer or hit enter: ')
            if name != '':
                self.cond['name'] = name.lower()
                
            country = raw_input('Search by country? y/[n]  ').strip().lower()
            if country == 'y':
                print 'Which country? (Hit enter for not selecting one)'
                print countries
                country = raw_input()
                if country != '' and int(country) in countries:
                    self.cond['country'] = countries[int(country)]
            # Printing the condtions
            print '-- Conditions --'
            print 'Price Range: ', self.cond['single_price']
            
            if self.cond['name'] == None:
                print 'Beer name contains:', self.cond['name']
            else: print 'Beer name contains: \"'+name+'\"'
            
            if 'country' in self.cond:
                print 'Country:', self.cond['country']
        except:
            print "Wrong input format..."
            self.askCondition()
    
    def exe(self):
        self.askCondition()
        coll = self.db.Price
        
        '''
        if kwargs is not None:
            for key, val in kwargs.iteritems():
                cond[key] = val
        '''
        
        if self.cond['single_price'] is None and self.cond['name'] is None : 
            if 'country' in self.cond:
                cursor = coll.find({'Country': self.cond['country']})
            else:
                cursor = coll.find()
        elif self.cond['single_price'] is not None and self.cond['name'] is None:
            if 'country' in self.cond:
                cursor = coll.find( { 'Single': { '$gte': min(self.cond['single_price']), '$lte': max(self.cond['single_price']) } ,
                                     'Country': self.cond['country']} )
            else:
                cursor = coll.find( { 'Single': { '$gte': min(self.cond['single_price']), '$lte': max(self.cond['single_price']) } } )
        elif self.cond['single_price'] is None and self.cond['name'] is not None:
            if 'country' in self.cond:
                cursor = coll.find( { 'Name': { '$regex': '.*'+self.cond['name']+'.*' } ,
                                     'Country': self.cond['country']} )
            else:
                cursor = coll.find( { 'Name': { '$regex': '.*'+self.cond['name']+'.*' } } )
        else:
            if 'country' in self.cond:
                cursor = coll.find( { 'Single': { '$gte': min(self.cond['single_price']), '$lte': max(self.cond['single_price']) },
                                     'Name': { '$regex': '.*'+self.cond['name']+'.*' }, 'Country': self.cond['country'] } )       
            else:
                cursor = coll.find( { 'Single': { '$gte': min(self.cond['single_price']), '$lte': max(self.cond['single_price']) },
                                     'Name': { '$regex': '.*'+self.cond['name']+'.*' } } )       
        return cursor
    
    def showResult(self, cursor):
        cursor = cursor
        data = {}        
        i = 0
        print "Number of result:", cursor.count()
        for result in cursor:
            for key in result.keys():
                if i == 0:
                    data[key] = [result[key]]
                else:
                    data[key].append(result[key])
            i+=1
        data['Name'] = [w.title().replace('Ipa','IPA') for w in data['Name']]
        frame = pd.DataFrame(data)
        return frame.ix[:,:-1]

In [ ]:
#This class allows users to search by a range of ABV and/or by name. While the majority of ABVs in this table lie 
#between 5%-7%, the entire range is from 0.1% to about 15%. This data set was taken from craft beer analytics. 

class ABV(Base):
    
    def askCondition(self):
        self.cond = {'abv': None, 'name': None}
        abv = raw_input('Enter an ABV range or hit enter: (e.g. 0, 2.29 ) ')
        
        try: 
            if abv != '':
                self.cond['abv'] = map(float, abv.replace(' ', '').split(','))            
            
            name = raw_input('Enter keyword of beer or hit enter: ')
            if name != '':
                self.cond['name'] = name.lower()
                
            
            print '-- Conditions --'
            print 'ABV Range: ', self.cond['abv']
            if self.cond['name'] == None:
                print 'Beer name contains:', self.cond['name']
            else: print 'Beer name contains: \"'+name+'\"'
        except:
            print "Wrong input format..."
            self.condition()
    
    def exe(self):
        self.askCondition()
        coll = self.db.ABV
        
        '''
        if kwargs is not None:
            for key, val in kwargs.iteritems():
                cond[key] = val
        '''
        
        if self.cond['abv'] is None and self.cond['name'] is None : 
            cursor = coll.find()
        elif self.cond['abv'] is not None and self.cond['name'] is None:
            cursor = coll.find( { 'ABV': { '$gte': min(self.cond['abv']), '$lte': max(self.cond['abv']) } } )
        elif self.cond['abv'] is None and self.cond['name'] is not None:
            cursor = coll.find( { 'BEER': { '$regex': '.*'+self.cond['name']+'.*' } } )
        else:
            cursor = coll.find( { 'ABV': { '$gte': min(self.cond['abv']), '$lte': max(self.cond['abv']) },
                                    'BEER': { '$regex': '.*'+self.cond['name']+'.*' } } )    
        return cursor
    
    def showResult(self, cursor):
        cursor = cursor
        print "Number of result:", cursor.count()
        data = {}        
        i = 0
        for result in cursor:
            for key in result.keys():
                if i == 0:
                    data[key] = [result[key]]
                else:
                    data[key].append(result[key])
            i+=1
        data['BEER'] = [w.title().replace('Ipa','IPA') for w in data['BEER']]
        frame = pd.DataFrame(data)
        return frame.ix[:, :-1]

In [19]:
#This last class allows users to search by name and/or rating (0-100). The data taken for this was taken from 
#ratebeer.com as collected by craft beer analytics. 

class Rating(Base):
    
    def askCondition(self):
        self.cond = {'rating': None, 'name': None}
        rating = raw_input('Enter rating range or hit enter: (e.g. 85, 100 ) ')
                
        try: 
            if rating != '':
                self.cond['rating'] = map(float, rating.replace(' ', '').split(','))
                if len(self.cond['rating']) != 2:
                    raise TypeError         
            
            name = raw_input('Enter keyword of beer or hit enter: ')
            if name != '':
                self.cond['name'] = name.lower()
                
            print '-- Conditions --'
            print 'Rating range: ', self.cond['rating']
            
            if self.cond['name'] == None:
                print 'Beer name contains:', self.cond['name']
            else: print 'Beer name contains: \"'+name+'\"'

        except:
            print "Wrong input format..."
            self.askCondition()
    
    def exe(self):
        self.askCondition()
        coll = self.db.Rating
        
        if self.cond['rating'] is None and self.cond['name'] is None : 
            cursor = coll.find()
        elif self.cond['rating'] is not None and self.cond['name'] is None:
            cursor = coll.find( { 'Rating': { '$gte': min(self.cond['rating']), '$lte': max(self.cond['rating']) } } )
        elif self.cond['rating'] is None and self.cond['name'] is not None:
            cursor = coll.find( { 'Name': { '$regex': '.*'+self.cond['name']+'.*' } } )
        else:
            cursor = coll.find( { 'rating': { '$gte': min(self.cond['rating']), '$lte': max(self.cond['rating']) },
                                     'Name': { '$regex': '.*'+self.cond['name']+'.*' } } )       
        return cursor
    
    def showResult(self, cursor):
        cursor = cursor
        data = {}        
        i = 0
        print "Number of result:", cursor.count()
        for result in cursor:
            for key in result.keys():
                if i == 0:
                    data[key] = [result[key]]
                else:
                    data[key].append(result[key])
            i+=1
        
        data['Name'] = [w.title().replace('Ipa','IPA') for w in data['Name']]
        frame = pd.DataFrame(data)
        return frame.ix[:,:-1]

In [ ]:
#This final class is the interface built for users to interact and search through the database. 
#Once run, the database will prompt you to pick the specific class you wish to interact with and then 
#allow input for parameters. 
class Query:
    
    def __init__(self):
        stop = False
        while stop != True:
            opt = raw_input('Search by (1)price (2)ABV (3)Rating (q)Quit: ').strip().lower()
            
            if opt in ('1', '2', '3', 'q'):
                if opt == 'q':
                    sure = raw_input('Quit? y/[n]  ')
                    if sure.lower() == 'y':
                        print 'End of Searching.'
                        break
                    else: continue
                elif int(opt) == 1:
                    result = self.priceQuery()
                elif int(opt) == 2:
                    result = self.abvQuery()
                elif int(opt) == 3:
                    result = self.ratingQuery()
            else: continue
            
            print result
            print '-----------------------'

    def priceQuery(self):
        p = Price()
        cursor = p.exe()
        result = p.showResult(cursor)
        return result
    def abvQuery(self):
        a = ABV()
        cursor = a.exe()
        result = a.showResult(cursor)
        return result
    
    def ratingQuery(self):
        r = Rating()
        cursor = r.exe()
        result = r.showResult(cursor)
        return result

In [ ]:
Query()